In [1]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps #Machine limits for floating point types.
from numpy import log2 as log

In [2]:
Alt = 'Yes,Yes,No,Yes,Yes,No,No,No,No,Yes,No,Yes'.split(',')
Bar = 'No,No,Yes,No,No,Yes,Yes,No,Yes,Yes,No,Yes'.split(',')
Fri = 'No,No,No,Yes,Yes,No,No,No,Yes,Yes,No,Yes'.split(',')
Hun = 'Yes,Yes,No,Yes,No,Yes,No,Yes,No,Yes,No,Yes'.split(',')
Pat = 'Some,Full,Some,Full,Full,Some,None,Some,Full,Full,None,Full'.split(',')
Price = '$$$,$,$,$,$$$,$$,$,$$,$,$$$,$,$'.split(',')
Rain = 'No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No'.split(',')
Res = 'Yes,No,No,No,Yes,Yes,No,Yes,No,Yes,No,No'.split(',')
Type = 'French,Thai,Burger,Thai,French,Italian,Burger,Thai,Burger,Italian,Thai,Burger'.split(',')
Est = '0-10,30-60,0-10,10-30,>60,0-10,0-10,0-10,>60,10-30,0-10,30-60'.split(',')
Willwait = 'Yes,No,Yes,Yes,No,Yes,No,Yes,No,No,No,Yes'.split(',')

In [3]:
dataset = {'Alt':Alt, 'Bar':Bar, 'Fri':Fri, 'Hun':Hun, 'Pat':Pat, 'Price':Price, 'Rain':Rain, 'Res':Res, 'Type':Type, 'Est':Est, 'Willwait':Willwait}

In [4]:
df = pd.DataFrame(dataset)
df

,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Willwait
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,No,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No
5,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,0-10,Yes
6,No,Yes,No,No,None,$,Yes,No,Burger,0-10,No
7,No,No,No,Yes,Some,$$,Yes,Yes,Thai,0-10,Yes
8,No,Yes,Yes,No,Full,$,Yes,No,Burger,>60,No
9,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,10-30,No


In [5]:
def entropy_dataset(df):
    Class = df.keys()[-1]
    variables = df[Class].unique()
    entropy = 0
    for variable in variables:
        fraction = df[Class].value_counts()[variable]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

def entropy(fraction):
    return -(fraction*log2(fraction) + (1-fraction)*log2(1-fraction))

In [6]:
entropy_dataset(df)

1.0

In [7]:
df['Pat'][df['Pat']=='Full'][df['Willwait']=='Yes']

3     Full
11    Full
Name: Pat, dtype: object

In [8]:
subtable = df[df['Pat'] == 'Full'].reset_index(drop=True)
clValue,counts = np.unique(subtable['Pat'],return_counts=True) 
clValue

array(['Full'], dtype=object)

In [9]:
counts

array([6])

In [10]:
def entropy_attribute(df, attribute):
    Class = df.keys()[-1]
    variables = df[attribute].unique()
    target_variables = df[Class].unique()
    entropy_final = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class]==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy_final += entropy*fraction2
    return abs(entropy_final)


In [11]:
def find_max_gain(df):
    IG = []
    for i in df.keys()[:-1]:
        IG.append(entropy_dataset(df) - entropy_attribute(df, i))
    #print("IG:",IG)
    return df.keys()[:-1][np.argmax(IG)]


In [12]:
def sub_table(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

In [15]:
def build_tree(df):
    Class = df.keys()[-1]
    node = find_max_gain(df)
    #print("max gain node: ", node)
    values = df[node].unique()
    #print("values of",node," ",values)
    try:
        tree
    except:
        tree = {}
    try:
        tree[node]
    except:
        tree[node] = {}
    for value in values:
        subtable = sub_table(df, node, value)
        #print(subtable)
        clValue,counts = np.unique(subtable[Class],return_counts=True)
        #print(clValue, " ", counts)
        if(len(counts) == 1):
            #print(tree)
            tree[node][value] = clValue[0]
            #print(tree)
        else:
            tree[node][value] = build_tree(subtable)
    return tree
        

In [16]:
tree = {}
tree = build_tree(df)

In [17]:
import pprint

In [18]:
pprint.pprint(tree)

{'Pat': {'Full': {'Hun': {'No': 'No',
                          'Yes': {'Type': {'Burger': 'Yes',
                                           'Italian': 'No',
                                           'Thai': {'Fri': {'No': 'No',
                                                            'Yes': 'Yes'}}}}}},
         'None': 'No',
         'Some': 'Yes'}}
